In [46]:
import os

In [47]:
from dotenv import load_dotenv
load_dotenv()

# Access the API key (this will not expose the key in your notebook)
google_api_key = os.getenv("GOOGLE_API_KEY")

# Verify it loaded correctly without exposing the key
if google_api_key:
    print("Gemini API key loaded successfully!")
else:
    print("Failed to load Gemini API key from .env file!")

Gemini API key loaded successfully!


In [48]:
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage
from typing import Annotated


In [49]:
# tool create

@tool
def multiply(a:int, b:int):
    """Given 2 numbers a & b this tool returns their product"""
    return a * b

In [50]:
print(multiply.invoke({'a':5,'b':9}))

45


In [51]:
multiply.name

'multiply'

In [52]:
multiply.description

'Given 2 numbers a & b this tool returns their product'

In [53]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

## Tool binding

In [54]:

llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-1.5-pro")
llm_with_tools = llm.bind_tools([multiply])

## Tool Calling

In [55]:
llm_with_tools.invoke("Hi! How are you ?")

AIMessage(content="I'm doing well, thank you for asking! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-85428235-7012-4f2e-b418-d0604e87c3c4-0', usage_metadata={'input_tokens': 23, 'output_tokens': 19, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})

In [56]:
query = HumanMessage('Can you multiply 8 with 89')

In [57]:
messages = [query]

In [58]:
messages

[HumanMessage(content='Can you multiply 8 with 89', additional_kwargs={}, response_metadata={})]

## Tool Execution

In [59]:
result = llm_with_tools.invoke(messages)


In [60]:
messages.append(result)

In [61]:
messages

[HumanMessage(content='Can you multiply 8 with 89', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 8.0, "b": 89.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-775a564b-5404-4ade-b8d9-ba95061c3c15-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8.0, 'b': 89.0}, 'id': 'e0aa9eee-04fd-4d96-9240-f1a28abc18f6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 26, 'output_tokens': 3, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})]

In [62]:
tool_result = multiply.invoke(result.tool_calls[0])

In [63]:
messages.append(tool_result)

In [64]:
messages

[HumanMessage(content='Can you multiply 8 with 89', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 8.0, "b": 89.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-775a564b-5404-4ade-b8d9-ba95061c3c15-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8.0, 'b': 89.0}, 'id': 'e0aa9eee-04fd-4d96-9240-f1a28abc18f6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 26, 'output_tokens': 3, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='712', name='multiply', tool_call_id='e0aa9eee-04fd-4d96-9240-f1a28abc18f6')]

In [65]:
llm_with_tools.invoke(messages).content

'The product of 8 and 89 is 712.'